<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer-compare/notebooks/transformer_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install requirements

In [1]:
! pip install transformers datasets
! pip install rouge-score nltk
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.3 MB/s 
     |████████████████████████████████| 451 kB 42.8 MB/s 
     |████████████████████████████████| 182 kB 61.9 MB/s 
     |████████████████████████████████| 7.6 MB 56.9 MB/s 
     |████████████████████████████████| 115 kB 65.3 MB/s 
     |████████████████████████████████| 212 kB 63.0 MB/s 
     |████████████████████████████████| 127 kB 60.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=cc8b1f84262454952da23f7dd336fd50acde0fe6023f6dfb0d3dbe02e6d326d8
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3

#Set parameters

In [41]:
model_checkpoint = 'sshleifer/distilbart-cnn-12-6'
dataset_name = 'xsum'
metric_name = 'rouge'

# Loading data

In [10]:
import transformers

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset(dataset_name)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [33]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5, random_seed=36):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    random.seed(random_seed)
    picks = random.sample(range(len(dataset)), num_examples)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
      if isinstance(typ, datasets.ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
    # To make it display entire document
    display(HTML(df.to_html()))

In [34]:
df = show_random_elements(raw_datasets["train"])


#Load metric

In [42]:
from datasets import load_metric

metric = load_metric(metric_name)

# Pre-process data

In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [49]:
prefix_models = ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]
if model_checkpoint in prefix_models:
    prefix = "summarize: "

In [47]:
model_checkpoint

'sshleifer/distilbart-cnn-12-6'

In [51]:
max_input_length = None
max_target_length = None

def preprocess_function(examples):
  if model_checkpoint in prefix_models:
    inputs = [prefix + doc for doc in examples["document"]]
  # If max input length provided, use it. 
  if max_input_length:
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  else:
    model_inputs = tokenizer(inputs, truncation=True)

  # Setup the tokenizer for targets #TODO: see if 
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(
          examples["summary"], max_length=max_target_length, truncation=True
      )

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs